# Phantom Airlines IROPS - Crew Ranking Model

This notebook implements an ML-powered crew ranking model for **One-Click Recovery**:

- **Feature Store**: Crew qualification, availability, and fatigue features
- **Model Registry**: Gradient boosting model for crew fit scoring
- **Model Observability**: Acceptance rate tracking and model drift detection

## Business Context
When a flight needs a replacement crew member, the system must rapidly identify and rank
available candidates based on qualification, proximity, fatigue levels, and historical
acceptance patterns. This reduces the 12-minute manual search to instant batch notifications.

## 1. Environment Setup

In [ ]:
import os
import warnings
warnings.filterwarnings('ignore')

from snowflake.snowpark import Session
from snowflake.snowpark import functions as F
from snowflake.snowpark.types import *

from snowflake.ml.feature_store import (
    FeatureStore,
    FeatureView,
    Entity,
    CreationMode
)
from snowflake.ml.registry import Registry
from snowflake.ml.modeling.preprocessing import StandardScaler, LabelEncoder
from snowflake.ml.modeling.pipeline import Pipeline
from snowflake.ml.modeling.lightgbm import LGBMClassifier
from snowflake.ml.modeling.metrics import (
    accuracy_score,
    precision_score,
    recall_score,
    roc_auc_score
)

import pandas as pd
import numpy as np

In [ ]:
connection_name = os.getenv("SNOWFLAKE_CONNECTION_NAME") or "default"
session = Session.builder.config("connection_name", connection_name).create()

DATABASE = os.getenv("IROPS_DATABASE", "PHANTOM_IROPS")
WAREHOUSE = os.getenv("IROPS_WAREHOUSE", "PHANTOM_IROPS_WH")

session.use_database(DATABASE)
session.use_warehouse(WAREHOUSE)

print(f"Connected to: {session.get_current_account()}")
print(f"Database: {DATABASE}")

## 2. Feature Store - Crew Features

In [ ]:
session.sql("CREATE SCHEMA IF NOT EXISTS FEATURE_STORE").collect()

fs = FeatureStore(
    session=session,
    database=DATABASE,
    name="FEATURE_STORE",
    default_warehouse=WAREHOUSE,
    creation_mode=CreationMode.CREATE_IF_NOT_EXIST
)

crew_entity = Entity(
    name="CREW_MEMBER",
    join_keys=["CREW_ID"],
    desc="Individual crew member for ranking and assignment"
)

try:
    fs.register_entity(crew_entity)
except:
    pass

print(f"Feature Store initialized with CREW_MEMBER entity")

### 2.1 Crew Profile Features

In [ ]:
crew_profile_query = f"""
SELECT 
    cm.CREW_ID,
    CURRENT_TIMESTAMP() AS FEATURE_TIMESTAMP,
    cm.CREW_TYPE,
    cm.BASE_AIRPORT,
    cm.SENIORITY_NUMBER,
    cm.YEARS_OF_SERVICE,
    cm.TOTAL_FLIGHT_HOURS,
    ARRAY_SIZE(SPLIT(cm.QUALIFIED_AIRCRAFT_TYPES, ',')) AS NUM_AIRCRAFT_QUALIFICATIONS,
    cm.AVAILABILITY_STATUS,
    cm.CONTRACT_TYPE,
    CASE 
        WHEN cm.SENIORITY_NUMBER < 5000 THEN 'SENIOR'
        WHEN cm.SENIORITY_NUMBER < 15000 THEN 'MID_LEVEL'
        ELSE 'JUNIOR'
    END AS SENIORITY_TIER,
    CASE 
        WHEN cm.YEARS_OF_SERVICE > 15 THEN 'VETERAN'
        WHEN cm.YEARS_OF_SERVICE > 5 THEN 'EXPERIENCED'
        ELSE 'NEW'
    END AS EXPERIENCE_TIER
FROM {DATABASE}.RAW.CREW_MEMBERS cm
"""

crew_profile_df = session.sql(crew_profile_query)

crew_profile_fv = FeatureView(
    name="CREW_PROFILE_FEATURES",
    entities=[crew_entity],
    feature_df=crew_profile_df,
    timestamp_col="FEATURE_TIMESTAMP",
    refresh_freq="1 hour",
    desc="Crew member profile and qualification features"
)

profile_fv = fs.register_feature_view(
    feature_view=crew_profile_fv,
    version="v1",
    block=True
)

print(f"Registered: {profile_fv.name} v{profile_fv.version}")

### 2.2 Crew Fatigue Features (FAA Part 117 Compliance)

In [ ]:
crew_fatigue_query = f"""
WITH duty_metrics AS (
    SELECT 
        CREW_ID,
        SUM(CASE WHEN DUTY_DATE >= DATEADD('day', -1, CURRENT_DATE()) THEN DUTY_HOURS ELSE 0 END) AS DUTY_HOURS_24H,
        SUM(CASE WHEN DUTY_DATE >= DATEADD('day', -7, CURRENT_DATE()) THEN DUTY_HOURS ELSE 0 END) AS DUTY_HOURS_7D,
        SUM(CASE WHEN DUTY_DATE >= DATEADD('day', -28, CURRENT_DATE()) THEN DUTY_HOURS ELSE 0 END) AS DUTY_HOURS_28D,
        SUM(CASE WHEN DUTY_DATE >= DATEADD('day', -7, CURRENT_DATE()) THEN FLIGHT_TIME_HOURS ELSE 0 END) AS FLIGHT_HOURS_7D,
        MAX(CUMULATIVE_MONTHLY_HOURS) AS MONTHLY_HOURS_USED,
        COUNT(DISTINCT CASE WHEN DUTY_DATE >= DATEADD('day', -7, CURRENT_DATE()) THEN DUTY_DATE END) AS DUTY_DAYS_7D,
        MAX(DUTY_DATE) AS LAST_DUTY_DATE
    FROM {DATABASE}.RAW.CREW_DUTY_LOG
    GROUP BY CREW_ID
)
SELECT 
    CREW_ID,
    CURRENT_TIMESTAMP() AS FEATURE_TIMESTAMP,
    COALESCE(DUTY_HOURS_24H, 0) AS DUTY_HOURS_24H,
    COALESCE(DUTY_HOURS_7D, 0) AS DUTY_HOURS_7D,
    COALESCE(DUTY_HOURS_28D, 0) AS DUTY_HOURS_28D,
    COALESCE(FLIGHT_HOURS_7D, 0) AS FLIGHT_HOURS_7D,
    100 - COALESCE(MONTHLY_HOURS_USED, 0) AS MONTHLY_HOURS_REMAINING,
    COALESCE(DUTY_DAYS_7D, 0) AS DUTY_DAYS_7D,
    7 - COALESCE(DUTY_DAYS_7D, 0) AS REST_DAYS_7D,
    DATEDIFF('hour', LAST_DUTY_DATE, CURRENT_TIMESTAMP()) AS HOURS_SINCE_LAST_DUTY,
    CASE 
        WHEN DUTY_HOURS_24H > 10 THEN 'HIGH_FATIGUE'
        WHEN DUTY_HOURS_7D > 50 THEN 'MODERATE_FATIGUE'
        WHEN DUTY_DAYS_7D >= 6 THEN 'MODERATE_FATIGUE'
        ELSE 'LOW_FATIGUE'
    END AS FATIGUE_LEVEL,
    CASE WHEN DUTY_HOURS_24H < 14 AND MONTHLY_HOURS_USED < 100 THEN TRUE ELSE FALSE END AS FAA_117_COMPLIANT
FROM duty_metrics
"""

crew_fatigue_df = session.sql(crew_fatigue_query)

crew_fatigue_fv = FeatureView(
    name="CREW_FATIGUE_FEATURES",
    entities=[crew_entity],
    feature_df=crew_fatigue_df,
    timestamp_col="FEATURE_TIMESTAMP",
    refresh_freq="15 minutes",
    desc="Crew fatigue metrics for FAA Part 117 compliance"
)

fatigue_fv = fs.register_feature_view(
    feature_view=crew_fatigue_fv,
    version="v1",
    block=True
)

print(f"Registered: {fatigue_fv.name} v{fatigue_fv.version}")

### 2.3 Crew Historical Performance Features

In [ ]:
crew_history_query = f"""
WITH assignment_stats AS (
    SELECT 
        ca.CREW_ID,
        COUNT(*) AS TOTAL_RECOVERY_OFFERS,
        SUM(CASE WHEN ca.RESPONSE_STATUS = 'ACCEPTED' THEN 1 ELSE 0 END) AS ACCEPTED_COUNT,
        SUM(CASE WHEN ca.RESPONSE_STATUS = 'DECLINED' THEN 1 ELSE 0 END) AS DECLINED_COUNT,
        SUM(CASE WHEN ca.RESPONSE_STATUS = 'TIMEOUT' THEN 1 ELSE 0 END) AS TIMEOUT_COUNT,
        AVG(DATEDIFF('minute', ca.NOTIFICATION_SENT_AT, ca.RESPONSE_AT)) AS AVG_RESPONSE_TIME_MIN,
        SUM(CASE WHEN ca.RESPONSE_STATUS = 'ACCEPTED' AND ca.NOTIFICATION_SENT_AT >= DATEADD('day', -30, CURRENT_DATE()) THEN 1 ELSE 0 END) AS ACCEPTED_30D
    FROM {DATABASE}.RAW.CREW_ASSIGNMENTS ca
    WHERE ca.ASSIGNMENT_SOURCE = 'RECOVERY'
    GROUP BY ca.CREW_ID
)
SELECT 
    CREW_ID,
    CURRENT_TIMESTAMP() AS FEATURE_TIMESTAMP,
    COALESCE(TOTAL_RECOVERY_OFFERS, 0) AS TOTAL_RECOVERY_OFFERS,
    COALESCE(ACCEPTED_COUNT, 0) AS RECOVERY_ACCEPTED_COUNT,
    COALESCE(DECLINED_COUNT, 0) AS RECOVERY_DECLINED_COUNT,
    COALESCE(TIMEOUT_COUNT, 0) AS RECOVERY_TIMEOUT_COUNT,
    CASE WHEN TOTAL_RECOVERY_OFFERS > 0 
        THEN ACCEPTED_COUNT / TOTAL_RECOVERY_OFFERS 
        ELSE 0.5 
    END AS HISTORICAL_ACCEPTANCE_RATE,
    COALESCE(AVG_RESPONSE_TIME_MIN, 5) AS AVG_RESPONSE_TIME_MIN,
    COALESCE(ACCEPTED_30D, 0) AS ACCEPTED_LAST_30D,
    CASE 
        WHEN ACCEPTED_COUNT / NULLIF(TOTAL_RECOVERY_OFFERS, 0) > 0.8 THEN 'RELIABLE'
        WHEN ACCEPTED_COUNT / NULLIF(TOTAL_RECOVERY_OFFERS, 0) > 0.5 THEN 'MODERATE'
        ELSE 'UNRELIABLE'
    END AS RELIABILITY_TIER
FROM assignment_stats
"""

crew_history_df = session.sql(crew_history_query)

crew_history_fv = FeatureView(
    name="CREW_HISTORY_FEATURES",
    entities=[crew_entity],
    feature_df=crew_history_df,
    timestamp_col="FEATURE_TIMESTAMP",
    refresh_freq="1 day",
    desc="Historical crew recovery assignment performance"
)

history_fv = fs.register_feature_view(
    feature_view=crew_history_fv,
    version="v1",
    block=True
)

print(f"Registered: {history_fv.name} v{history_fv.version}")

## 3. Training Data - Historical Acceptance Outcomes

In [ ]:
training_query = f"""
SELECT 
    ca.CREW_ID,
    ca.NOTIFICATION_SENT_AT AS LABEL_TIMESTAMP,
    cm.BASE_AIRPORT = f.ORIGIN AS IS_SAME_BASE,
    CONTAINS(cm.QUALIFIED_AIRCRAFT_TYPES, f.AIRCRAFT_TYPE_CODE) AS IS_TYPE_QUALIFIED,
    cm.SENIORITY_NUMBER,
    cm.YEARS_OF_SERVICE,
    CASE WHEN ca.RESPONSE_STATUS = 'ACCEPTED' THEN 1 ELSE 0 END AS WAS_ACCEPTED
FROM {DATABASE}.RAW.CREW_ASSIGNMENTS ca
JOIN {DATABASE}.RAW.CREW_MEMBERS cm ON ca.CREW_ID = cm.CREW_ID
JOIN {DATABASE}.RAW.FLIGHTS f ON ca.FLIGHT_ID = f.FLIGHT_ID
WHERE ca.ASSIGNMENT_SOURCE = 'RECOVERY'
  AND ca.RESPONSE_STATUS IN ('ACCEPTED', 'DECLINED', 'TIMEOUT')
"""

spine_df = session.sql(training_query)
print(f"Training spine rows: {spine_df.count()}")

In [ ]:
training_data = fs.generate_training_set(
    spine_df=spine_df,
    features=[
        profile_fv,
        fatigue_fv,
        history_fv
    ],
    spine_timestamp_col="LABEL_TIMESTAMP",
    spine_label_cols=["WAS_ACCEPTED"],
    include_feature_view_timestamp_col=False
)

training_df = training_data.to_snowpark_dataframe()
print(f"Training data with features: {training_df.count()} rows")
training_df.limit(5).show()

## 4. Model Training - LightGBM Classifier

In [ ]:
train_df, test_df = training_df.random_split([0.8, 0.2], seed=42)

print(f"Training set: {train_df.count()} rows")
print(f"Test set: {test_df.count()} rows")

In [ ]:
NUMERIC_COLS = [
    "SENIORITY_NUMBER",
    "YEARS_OF_SERVICE",
    "NUM_AIRCRAFT_QUALIFICATIONS",
    "DUTY_HOURS_24H",
    "DUTY_HOURS_7D",
    "DUTY_HOURS_28D",
    "FLIGHT_HOURS_7D",
    "MONTHLY_HOURS_REMAINING",
    "DUTY_DAYS_7D",
    "REST_DAYS_7D",
    "HOURS_SINCE_LAST_DUTY",
    "TOTAL_RECOVERY_OFFERS",
    "RECOVERY_ACCEPTED_COUNT",
    "HISTORICAL_ACCEPTANCE_RATE",
    "AVG_RESPONSE_TIME_MIN"
]

BOOLEAN_COLS = [
    "IS_SAME_BASE",
    "IS_TYPE_QUALIFIED",
    "FAA_117_COMPLIANT"
]

LABEL_COL = "WAS_ACCEPTED"
FEATURE_COLS = NUMERIC_COLS + BOOLEAN_COLS

In [ ]:
scaler = StandardScaler(
    input_cols=NUMERIC_COLS,
    output_cols=[f"{c}_SCALED" for c in NUMERIC_COLS]
)

lgbm_classifier = LGBMClassifier(
    input_cols=[f"{c}_SCALED" for c in NUMERIC_COLS] + BOOLEAN_COLS,
    label_cols=[LABEL_COL],
    output_cols=["PREDICTED_ACCEPTANCE"],
    n_estimators=100,
    max_depth=8,
    learning_rate=0.05,
    num_leaves=31,
    random_state=42
)

pipeline = Pipeline(steps=[
    ("scaler", scaler),
    ("classifier", lgbm_classifier)
])

print("Pipeline created with StandardScaler + LGBMClassifier")

In [ ]:
print("Training crew ranking model...")
pipeline.fit(train_df)
print("Model training complete!")

## 5. Model Evaluation

In [ ]:
predictions_df = pipeline.predict(test_df)

predictions_pd = predictions_df.select(
    LABEL_COL, 
    "PREDICTED_ACCEPTANCE"
).to_pandas()

y_true = predictions_pd[LABEL_COL]
y_pred = predictions_pd["PREDICTED_ACCEPTANCE"]

accuracy = accuracy_score(y_true=y_true, y_pred=y_pred)
precision = precision_score(y_true=y_true, y_pred=y_pred)
recall = recall_score(y_true=y_true, y_pred=y_pred)

metrics = {
    "accuracy": float(accuracy),
    "precision": float(precision),
    "recall": float(recall)
}

print("Model Performance Metrics:")
print(f"  Accuracy:  {accuracy:.4f}")
print(f"  Precision: {precision:.4f}")
print(f"  Recall:    {recall:.4f}")

## 6. Model Registry

In [ ]:
session.sql("CREATE SCHEMA IF NOT EXISTS ML_MODELS").collect()

registry = Registry(
    session=session,
    database_name=DATABASE,
    schema_name="ML_MODELS"
)

sample_input = train_df.select(FEATURE_COLS).limit(100)

model_version = registry.log_model(
    model=pipeline,
    model_name="CREW_RANKING_MODEL",
    version_name="V1",
    sample_input_data=sample_input,
    metrics=metrics,
    conda_dependencies=["scikit-learn", "lightgbm"],
    comment="LightGBM classifier for crew acceptance prediction. Used for One-Click Recovery ranking."
)

print(f"Model registered: {model_version.model_name} version {model_version.version_name}")

## 7. Model Observability

In [ ]:
observability_sql = f"""
CREATE TABLE IF NOT EXISTS {DATABASE}.ML_MODELS.CREW_RANKING_PREDICTIONS_LOG (
    PREDICTION_ID VARCHAR(50) DEFAULT UUID_STRING(),
    PREDICTION_TIMESTAMP TIMESTAMP_NTZ DEFAULT CURRENT_TIMESTAMP(),
    MODEL_VERSION VARCHAR(20),
    FLIGHT_ID VARCHAR(50),
    CREW_ID VARCHAR(50),
    ML_FIT_SCORE FLOAT,
    CANDIDATE_RANK INTEGER,
    WAS_NOTIFIED BOOLEAN,
    ACTUAL_RESPONSE VARCHAR(20),
    RESPONSE_TIME_MINUTES INTEGER
)
"""
session.sql(observability_sql).collect()

monitor_sql = f"""
CREATE OR REPLACE VIEW {DATABASE}.ML_MODELS.CREW_RANKING_PERFORMANCE_MONITOR AS
SELECT 
    DATE_TRUNC('day', PREDICTION_TIMESTAMP) AS PREDICTION_DATE,
    MODEL_VERSION,
    COUNT(*) AS TOTAL_PREDICTIONS,
    COUNT(DISTINCT FLIGHT_ID) AS FLIGHTS_RECOVERED,
    AVG(ML_FIT_SCORE) AS AVG_FIT_SCORE,
    SUM(CASE WHEN ACTUAL_RESPONSE = 'ACCEPTED' THEN 1 ELSE 0 END) / 
        NULLIF(SUM(CASE WHEN WAS_NOTIFIED THEN 1 ELSE 0 END), 0) AS ACCEPTANCE_RATE,
    AVG(CASE WHEN ACTUAL_RESPONSE IS NOT NULL THEN RESPONSE_TIME_MINUTES END) AS AVG_RESPONSE_TIME
FROM {DATABASE}.ML_MODELS.CREW_RANKING_PREDICTIONS_LOG
WHERE ACTUAL_RESPONSE IS NOT NULL
GROUP BY DATE_TRUNC('day', PREDICTION_TIMESTAMP), MODEL_VERSION
"""
session.sql(monitor_sql).collect()

print("Crew ranking observability infrastructure created")

## 8. Crew Ranking Inference View

In [ ]:
fit_score_function_sql = f"""
CREATE OR REPLACE FUNCTION {DATABASE}.ML_MODELS.CALCULATE_CREW_FIT_SCORE(
    is_type_qualified BOOLEAN,
    is_same_base BOOLEAN,
    monthly_hours_remaining FLOAT,
    flight_hours_last_7_days FLOAT,
    seniority_number INTEGER,
    historical_acceptance_rate FLOAT,
    faa_compliant BOOLEAN
)
RETURNS FLOAT
LANGUAGE SQL
AS
$$
    CASE WHEN NOT faa_compliant THEN 0 ELSE
        (CASE WHEN is_type_qualified THEN 30 ELSE 0 END) +
        (CASE WHEN is_same_base THEN 25 ELSE 10 END) +
        (LEAST(20, monthly_hours_remaining * 0.4)) +
        (GREATEST(0, 10 - flight_hours_last_7_days * 0.3)) +
        (CASE WHEN seniority_number < 5000 THEN 5 ELSE 0 END) +
        (historical_acceptance_rate * 10)
    END
$$
"""
session.sql(fit_score_function_sql).collect()

print(f"Created: {DATABASE}.ML_MODELS.CALCULATE_CREW_FIT_SCORE function")

In [ ]:
ranking_view_sql = f"""
CREATE OR REPLACE VIEW {DATABASE}.ML_MODELS.CREW_CANDIDATE_RANKINGS AS
WITH flights_needing_crew AS (
    SELECT 
        f.FLIGHT_ID,
        f.FLIGHT_NUMBER,
        f.ORIGIN,
        f.DESTINATION,
        f.SCHEDULED_DEPARTURE_UTC,
        f.AIRCRAFT_TYPE_CODE,
        f.CAPTAIN_ID,
        f.FIRST_OFFICER_ID,
        CASE WHEN f.CAPTAIN_ID IS NULL THEN TRUE ELSE FALSE END AS NEEDS_CAPTAIN,
        CASE WHEN f.FIRST_OFFICER_ID IS NULL THEN TRUE ELSE FALSE END AS NEEDS_FIRST_OFFICER
    FROM {DATABASE}.ANALYTICS.MART_GOLDEN_RECORD f
    WHERE f.FLIGHT_DATE >= CURRENT_DATE()
      AND (f.CAPTAIN_ID IS NULL OR f.FIRST_OFFICER_ID IS NULL)
),
available_crew AS (
    SELECT 
        cm.CREW_ID,
        cm.FULL_NAME,
        cm.CREW_TYPE,
        cm.BASE_AIRPORT,
        cm.PHONE_NUMBER,
        cm.EMAIL,
        cm.QUALIFIED_AIRCRAFT_TYPES,
        cm.SENIORITY_NUMBER,
        COALESCE(cdl.MONTHLY_HOURS_REMAINING, 100) AS MONTHLY_HOURS_REMAINING,
        COALESCE(cdl.FLIGHT_HOURS_7D, 0) AS FLIGHT_HOURS_7D,
        COALESCE(ch.HISTORICAL_ACCEPTANCE_RATE, 0.5) AS HISTORICAL_ACCEPTANCE_RATE,
        cm.AVAILABILITY_STATUS = 'AVAILABLE' AS IS_AVAILABLE,
        COALESCE(cdl.MONTHLY_HOURS_REMAINING, 100) > 8 AS FAA_COMPLIANT
    FROM {DATABASE}.RAW.CREW_MEMBERS cm
    LEFT JOIN (
        SELECT CREW_ID, 
               100 - MAX(CUMULATIVE_MONTHLY_HOURS) AS MONTHLY_HOURS_REMAINING,
               SUM(CASE WHEN DUTY_DATE >= DATEADD('day', -7, CURRENT_DATE()) THEN FLIGHT_TIME_HOURS ELSE 0 END) AS FLIGHT_HOURS_7D
        FROM {DATABASE}.RAW.CREW_DUTY_LOG
        GROUP BY CREW_ID
    ) cdl ON cm.CREW_ID = cdl.CREW_ID
    LEFT JOIN (
        SELECT CREW_ID,
               SUM(CASE WHEN RESPONSE_STATUS = 'ACCEPTED' THEN 1 ELSE 0 END) / NULLIF(COUNT(*), 0) AS HISTORICAL_ACCEPTANCE_RATE
        FROM {DATABASE}.RAW.CREW_ASSIGNMENTS
        WHERE ASSIGNMENT_SOURCE = 'RECOVERY'
        GROUP BY CREW_ID
    ) ch ON cm.CREW_ID = ch.CREW_ID
    WHERE cm.AVAILABILITY_STATUS = 'AVAILABLE'
      AND cm.CREW_TYPE IN ('CAPTAIN', 'FIRST_OFFICER')
)
SELECT 
    f.FLIGHT_ID,
    f.FLIGHT_NUMBER,
    f.ORIGIN,
    f.DESTINATION,
    f.SCHEDULED_DEPARTURE_UTC,
    f.AIRCRAFT_TYPE_CODE,
    c.CREW_ID,
    c.FULL_NAME AS CREW_NAME,
    c.CREW_TYPE,
    c.BASE_AIRPORT,
    c.PHONE_NUMBER,
    c.EMAIL,
    CONTAINS(c.QUALIFIED_AIRCRAFT_TYPES, f.AIRCRAFT_TYPE_CODE) AS IS_TYPE_QUALIFIED,
    c.BASE_AIRPORT = f.ORIGIN AS IS_SAME_BASE,
    c.MONTHLY_HOURS_REMAINING,
    c.FLIGHT_HOURS_7D,
    c.SENIORITY_NUMBER,
    c.HISTORICAL_ACCEPTANCE_RATE,
    c.FAA_COMPLIANT,
    {DATABASE}.ML_MODELS.CALCULATE_CREW_FIT_SCORE(
        CONTAINS(c.QUALIFIED_AIRCRAFT_TYPES, f.AIRCRAFT_TYPE_CODE),
        c.BASE_AIRPORT = f.ORIGIN,
        c.MONTHLY_HOURS_REMAINING,
        c.FLIGHT_HOURS_7D,
        c.SENIORITY_NUMBER,
        c.HISTORICAL_ACCEPTANCE_RATE,
        c.FAA_COMPLIANT
    ) AS ML_FIT_SCORE,
    ROW_NUMBER() OVER (
        PARTITION BY f.FLIGHT_ID, c.CREW_TYPE 
        ORDER BY {DATABASE}.ML_MODELS.CALCULATE_CREW_FIT_SCORE(
            CONTAINS(c.QUALIFIED_AIRCRAFT_TYPES, f.AIRCRAFT_TYPE_CODE),
            c.BASE_AIRPORT = f.ORIGIN,
            c.MONTHLY_HOURS_REMAINING,
            c.FLIGHT_HOURS_7D,
            c.SENIORITY_NUMBER,
            c.HISTORICAL_ACCEPTANCE_RATE,
            c.FAA_COMPLIANT
        ) DESC
    ) AS CANDIDATE_RANK
FROM flights_needing_crew f
CROSS JOIN available_crew c
WHERE c.FAA_COMPLIANT
  AND (
    (f.NEEDS_CAPTAIN AND c.CREW_TYPE = 'CAPTAIN') OR
    (f.NEEDS_FIRST_OFFICER AND c.CREW_TYPE = 'FIRST_OFFICER')
  )
"""
session.sql(ranking_view_sql).collect()

print(f"Created: {DATABASE}.ML_MODELS.CREW_CANDIDATE_RANKINGS view")

## 9. Summary

In [ ]:
print("="*60)
print("CREW RANKING MODEL - DEPLOYMENT COMPLETE")
print("="*60)
print("\nFeature Store Artifacts:")
print("  - Entity: CREW_MEMBER")
print("  - Feature View: CREW_PROFILE_FEATURES v1")
print("  - Feature View: CREW_FATIGUE_FEATURES v1")
print("  - Feature View: CREW_HISTORY_FEATURES v1")
print("\nModel Registry:")
print(f"  - Model: CREW_RANKING_MODEL V1")
print(f"  - Algorithm: LightGBM Classifier")
print(f"  - Features: {len(FEATURE_COLS)}")
print(f"  - Accuracy: {accuracy:.4f}")
print("\nObservability:")
print(f"  - Predictions Log: {DATABASE}.ML_MODELS.CREW_RANKING_PREDICTIONS_LOG")
print(f"  - Performance Monitor: {DATABASE}.ML_MODELS.CREW_RANKING_PERFORMANCE_MONITOR")
print("\nInference:")
print(f"  - Function: {DATABASE}.ML_MODELS.CALCULATE_CREW_FIT_SCORE")
print(f"  - View: {DATABASE}.ML_MODELS.CREW_CANDIDATE_RANKINGS")
print("="*60)

In [ ]:
session.close()
print("Session closed.")